In [1]:
import numpy as np
import pandas as pd

In [2]:
f = open('new_user_artists_scaled.txt', 'r')

In [3]:
arr = []
for s in f:
    arr.append(s.split('\t'))

In [4]:
weights = []
for i in range(len(arr)):
    arr[i][0] = int(arr[i][0])
    arr[i][1] = int(arr[i][1])
    arr[i][2] = float(arr[i][2])
    weights.append(arr[i][2])

In [5]:
df = pd.DataFrame(arr, columns=['uid', 'iid', 'rate'])

In [6]:
df

,uid,iid,rate
0,2,51,3.733741
1,2,52,3.666446
2,2,53,3.654928
3,2,54,3.616898
4,2,55,3.563351
5,2,56,3.415189
6,2,57,3.402451
7,2,58,3.302770
8,2,59,3.278371
9,2,60,3.260839


In [7]:
rmin = min(df["rate"])
rmax = max(df["rate"])
rmid = (rmin + rmax) / 2.0

In [8]:
print(rmin)
print(rmax)
print(rmid)

0.271325
5.0
2.6356625


In [9]:
def Agreement(r1, r2):
    if ((r1 > rmid) and (r2 < rmid)) or ((r1 < rmid) and (r2 > rmid)):
        return False
    return True

In [10]:
def Proximity(r1,r2):
    if Agreement(r1,r2):
        absDist = abs(r1-r2)
    absDist = 2*abs(r1-r2)
    return ((2 * (rmax - rmin) + 1) - absDist)**2

In [11]:
def Impact(r1,r2):
     return (((abs(r1 - rmid) + 1)*(abs(r2 - rmid) + 1)) 
             if Agreement(r1,r2) 
             else (1.0 / ((abs(r1 - rmid) + 1) * (abs(r2 - rmid) + 1))))

In [12]:
def Popularity(arr, r1, r2, item):
    itemRating = []
    for i in range(len(arr)):
        if arr[i][1] == item:
            itemRating.append(arr[i][2])
                
    avg = np.mean(itemRating)
    return ((1 + (((r1 + r2)/2.0 - avg) ** 2)) 
            if (r1 > avg and r2 > avg) or (r1 < avg and r2 < avg) 
            else 1)

In [24]:
def PIP(arr, df, user1, user2):
    commonItem = {}
    
    u1_items = df.loc[df["uid"] == user1, "iid"]
    u2_items = df.loc[df["uid"] == user2, "iid"]
    for item in u1_items:
        if item in u2_items:
            commonItem[item] = 1
                     
    if len(commonItem)==0: return 0
       
    result = 0
    for item in u1_items:
        if item in u2_items:
            r1 = df.loc[(df["uid"] == user1)&(df["iid"] == item), "rate"].sum()
            r2 = df.loc[(df["uid"] == user2)&(df["iid"] == item), "rate"].sum()
            result += Proximity(r1, r2) * Impact(r1, r2) * Popularity(arr, r1, r2, item)

    return result

In [16]:
users = []
users = df["uid"]
users = set(users)
users = list(users)

In [18]:
len(users)

1885

In [19]:
PIP(arr, df, 2, 4)

2.811455
0.0


5.467365435604077

In [26]:
PIP_mat = np.zeros((len(users), len(users)))

for i in range(0, len(users)):
    print(i+1, "/", len(users))
    for j in range(i, len(users)):
        PIP_mat[i][j] = PIP(arr, df, users[i], users[j])
        PIP_mat[j][i] = PIP_mat[i][j]

1 / 1885
2 / 1885
3 / 1885
4 / 1885
5 / 1885
6 / 1885
7 / 1885
8 / 1885
9 / 1885
10 / 1885
11 / 1885
12 / 1885
13 / 1885
14 / 1885
15 / 1885
16 / 1885
17 / 1885
18 / 1885
19 / 1885
20 / 1885
21 / 1885
22 / 1885
23 / 1885
24 / 1885
25 / 1885
26 / 1885
27 / 1885
28 / 1885
29 / 1885
30 / 1885
31 / 1885
32 / 1885
33 / 1885
34 / 1885
35 / 1885
36 / 1885
37 / 1885
38 / 1885
39 / 1885
40 / 1885
41 / 1885
42 / 1885
43 / 1885
44 / 1885
45 / 1885
46 / 1885
47 / 1885
48 / 1885
49 / 1885
50 / 1885
51 / 1885
52 / 1885
53 / 1885
54 / 1885
55 / 1885
56 / 1885
57 / 1885
58 / 1885
59 / 1885
60 / 1885
61 / 1885
62 / 1885
63 / 1885
64 / 1885
65 / 1885
66 / 1885
67 / 1885
68 / 1885
69 / 1885
70 / 1885
71 / 1885
72 / 1885
73 / 1885
74 / 1885
75 / 1885
76 / 1885
77 / 1885
78 / 1885
79 / 1885
80 / 1885
81 / 1885
82 / 1885
83 / 1885
84 / 1885
85 / 1885
86 / 1885
87 / 1885
88 / 1885
89 / 1885
90 / 1885
91 / 1885
92 / 1885
93 / 1885
94 / 1885
95 / 1885
96 / 1885
97 / 1885
98 / 1885
99 / 1885
100 / 1885
101 / 18

756 / 1885
757 / 1885
758 / 1885
759 / 1885
760 / 1885
761 / 1885
762 / 1885
763 / 1885
764 / 1885
765 / 1885
766 / 1885
767 / 1885
768 / 1885
769 / 1885
770 / 1885
771 / 1885
772 / 1885
773 / 1885
774 / 1885
775 / 1885
776 / 1885
777 / 1885
778 / 1885
779 / 1885
780 / 1885
781 / 1885
782 / 1885
783 / 1885
784 / 1885
785 / 1885
786 / 1885
787 / 1885
788 / 1885
789 / 1885
790 / 1885
791 / 1885
792 / 1885
793 / 1885
794 / 1885
795 / 1885
796 / 1885
797 / 1885
798 / 1885
799 / 1885
800 / 1885
801 / 1885
802 / 1885
803 / 1885
804 / 1885
805 / 1885
806 / 1885
807 / 1885
808 / 1885
809 / 1885
810 / 1885
811 / 1885
812 / 1885
813 / 1885
814 / 1885
815 / 1885
816 / 1885
817 / 1885
818 / 1885
819 / 1885
820 / 1885
821 / 1885
822 / 1885
823 / 1885
824 / 1885
825 / 1885
826 / 1885
827 / 1885
828 / 1885
829 / 1885
830 / 1885
831 / 1885
832 / 1885
833 / 1885
834 / 1885
835 / 1885
836 / 1885
837 / 1885
838 / 1885
839 / 1885
840 / 1885
841 / 1885
842 / 1885
843 / 1885
844 / 1885
845 / 1885
846 / 1885

1460 / 1885
1461 / 1885
1462 / 1885
1463 / 1885
1464 / 1885
1465 / 1885
1466 / 1885
1467 / 1885
1468 / 1885
1469 / 1885
1470 / 1885
1471 / 1885
1472 / 1885
1473 / 1885
1474 / 1885
1475 / 1885
1476 / 1885
1477 / 1885
1478 / 1885
1479 / 1885
1480 / 1885
1481 / 1885
1482 / 1885
1483 / 1885
1484 / 1885
1485 / 1885
1486 / 1885
1487 / 1885
1488 / 1885
1489 / 1885
1490 / 1885
1491 / 1885
1492 / 1885
1493 / 1885
1494 / 1885
1495 / 1885
1496 / 1885
1497 / 1885
1498 / 1885
1499 / 1885
1500 / 1885
1501 / 1885
1502 / 1885
1503 / 1885
1504 / 1885
1505 / 1885
1506 / 1885
1507 / 1885
1508 / 1885
1509 / 1885
1510 / 1885
1511 / 1885
1512 / 1885
1513 / 1885
1514 / 1885
1515 / 1885
1516 / 1885
1517 / 1885
1518 / 1885
1519 / 1885
1520 / 1885
1521 / 1885
1522 / 1885
1523 / 1885
1524 / 1885
1525 / 1885
1526 / 1885
1527 / 1885
1528 / 1885
1529 / 1885
1530 / 1885
1531 / 1885
1532 / 1885
1533 / 1885
1534 / 1885
1535 / 1885
1536 / 1885
1537 / 1885
1538 / 1885
1539 / 1885
1540 / 1885
1541 / 1885
1542 / 1885
1543

In [27]:
PIP_mat

array([[0.00000000e+00, 1.81767963e+02, 5.46736544e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.81767963e+02, 0.00000000e+00, 1.71804968e+04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.46736544e+00, 1.71804968e+04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])